In [1]:
from read_data import read_data
import pandas as pd
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score

def data_preprocess(mode):
    behaviors = read_data(mode=mode, dataset='behaviors')
    behaviors['time1'] = behaviors['time'].apply(
        lambda x: time.mktime(time.strptime(x, '%m/%d/%Y %H:%M:%S %p'))).astype('int64')
    behaviors['time2'] = behaviors['time'].str[-2:]

    # impressions = train_behaviors['impressions']
    behaviors = behaviors[['user_id', 'time1', 'time2', 'impressions']]
    behaviors['user_id'] = behaviors['user_id'].map(lambda x: x.lstrip('U'))

    behaviors['time_AM'] = behaviors['time2'].apply(lambda x: 1 if x == 'AM' else 0).astype('int32')
    behaviors['time_PM'] = behaviors['time2'].apply(lambda x: 1 if x == 'PM' else 0).astype('int32')
    behaviors = behaviors[['user_id', 'time1', 'time_AM', 'time_PM', 'impressions']]
    behaviors = behaviors['impressions'].str.split(' ', expand=True).stack().reset_index(level=0).set_index('level_0').rename(columns={0:'impressions'}).join(behaviors.drop('impressions', axis=1))
    behaviors['impressions'] = behaviors['impressions'].map(lambda x: x.lstrip('N'))
    if mode == 'train' or mode == 'val':
        behaviors['impressions'], behaviors['lables'] = behaviors['impressions'].str.split('-').str
        behaviors['lables'] = behaviors['lables'].astype('int32')
    # train_behaviors = pd.get_dummies(train_behaviors)
    return behaviors


In [2]:
df = data_preprocess('train')
df.head()

C:\Users\19853\AppData\Local\Temp\ipykernel_17516\1709285470.py:24: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  behaviors['impressions'], behaviors['lables'] = behaviors['impressions'].str.split('-').str


,impressions,user_id,time1,time_AM,time_PM,lables
0,78206,87243,1573356654,1,0,0
0,26368,87243,1573356654,1,0,0
0,7578,87243,1573356654,1,0,0
0,58592,87243,1573356654,1,0,0
0,19858,87243,1573356654,1,0,0


In [3]:
lr = LogisticRegression()
lr.fit(df.iloc[:, :-1], df.iloc[:, -1])

LogisticRegression()

In [4]:
df = data_preprocess('val')
lr_pred = lr.predict(df.iloc[:, :-1])
roc_auc_score(df.iloc[:, -1], lr_pred)

C:\Users\19853\AppData\Local\Temp\ipykernel_17516\1709285470.py:24: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  behaviors['impressions'], behaviors['lables'] = behaviors['impressions'].str.split('-').str


0.5

In [6]:
accuracy_score(df.iloc[:, -1], lr_pred)

0.9591059182118364

In [7]:
mean_squared_error(df.iloc[:, -1], lr_pred)

0.040894081788163575